<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1:Sivert Escaff
- Nombre de alumno 2:Rodrigo Montecino


### **Link de repositorio de GitHub:** [Insertar Repositorio](https://github.com/...../)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [22]:
import pandas as pd


df = pd.read_csv('sales.csv')


df.head()


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [23]:
df.columns

Index(['id', 'date', 'city', 'lat', 'long', 'pop', 'shop', 'brand',
       'container', 'capacity', 'price', 'quantity'],
      dtype='object')

## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [50]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Load the sales data
sales_data = pd.read_csv('sales.csv')

def extract_date_parts(df):
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y', errors='coerce')  # Adding format specification
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    return df[['day', 'month', 'year']]



seed = 42

# Splitting the data
train_data, temp_data = train_test_split(sales_data, test_size=0.3, random_state=seed)
validation_data, test_data = train_test_split(temp_data, test_size=(1/3), random_state=seed)

# Numeric and categorical transformers
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = ['city', 'shop', 'brand', 'container', 'capacity']
categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Combine transformers
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('date', FunctionTransformer(extract_date_parts, validate=False), ['date'])
    ],
    remainder='drop'
)


pipeline_dummy = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])


pipeline_dummy.fit(train_data, train_data['quantity'])


predictions = pipeline_dummy.predict(validation_data)
mae_dummy = mean_absolute_error(validation_data['quantity'], predictions)
print(f'Mean Absolute Error with DummyRegressor: {mae_dummy}')


with open('dummy_regressor_model.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)


pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])


pipeline_xgb.fit(train_data, train_data['quantity'])


predictions_xgb = pipeline_xgb.predict(validation_data)
mae_xgb = mean_absolute_error(validation_data['quantity'], predictions_xgb)
print(f'Mean Absolute Error with XGBRegressor: {mae_xgb}')


with open('xgb_regressor_model.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)


Mean Absolute Error with DummyRegressor: 13298.497767341096
Mean Absolute Error with XGBRegressor: 173.83421263256622


## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [55]:

# existe un problema con usar xgboost por la version de python 
feature_names = [
    'lat', 'long', 'pop', 'price', 'quantity', 
    'city_Athens', 'city_Other', 
    'shop_shop_1', 'shop_shop_2', 
    'brand_kinder-cola', 'brand_adult-cola', 
    'container_glass', 'container_plastic', 'container_can',  
    'capacity_500ml', 'capacity_1.5lt', 'capacity_330ml',  
    'day', 'month', 'year' 
]


price_index = feature_names.index('price')  


constraints = [0] * len(feature_names)
constraints[price_index] = -1 


pipeline_xgb_mono = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=tuple(constraints)))
])

# Train the pipeline
pipeline_xgb_mono.fit(train_data, train_data['quantity'])

# Evaluate the model
predictions_xgb_mono = pipeline_xgb_mono.predict(validation_data)
mae_xgb_mono = mean_absolute_error(validation_data['quantity'], predictions_xgb_mono)
print(f'Mean Absolute Error with XGBRegressor with Monotonic Constraint: {mae_xgb_mono}')

# Save the model
with open('xgb_regressor_mono_model.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb_mono, f)



Mean Absolute Error with XGBRegressor with Monotonic Constraint: 166.7151864638351


## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [62]:


X_train = train_data.drop(columns=['quantity'])
y_train = train_data['quantity']
X_valid = validation_data.drop(columns=['quantity'])
y_valid = validation_data['quantity']


In [64]:
import logging


In [65]:
logging.basicConfig(level=logging.INFO)


In [66]:
import optuna
import pickle
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
import numpy as np



import logging

def objective(trial):
    try:
        
        params = {
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
            "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "max_leaves": trial.suggest_int("max_leaves", 0, 100),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
            "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
            "random_state": 42  
        }
        
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        regressor = XGBRegressor(**params)

        pipeline = Pipeline([
            ('encoder', encoder),
            ('regressor', regressor)
        ])

        pipeline.fit(X_train, y_train)
        preds = pipeline.predict(X_valid)
        mae = mean_absolute_error(y_valid, preds)

        logging.info(f"Trial completed: MAE={mae}")
        trial.set_user_attr("best_pipeline", pipeline)  
        return mae
    except Exception as e:
        logging.error(f"Error during trial: {str(e)}")
        return float('inf') 



sampler = optuna.samplers.TPESampler(seed=42) 
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100, timeout=300)

if "best_pipeline" in study.user_attrs:
    best_pipeline = study.user_attrs["best_pipeline"]
    print(f"Best MAE: {study.best_value}")
    print("Best Hyperparameters:", study.best_params)
    

    with open('best_model.pkl', 'wb') as f:
        pickle.dump(best_pipeline, f)
else:
    print("No best pipeline found. Check the trial setup and execution.")


INFO:root:Trial completed: MAE=5803.723809245446
INFO:root:Trial completed: MAE=7093.4036440577465
INFO:root:Trial completed: MAE=7522.573399380659
INFO:root:Trial completed: MAE=7365.327604952632
INFO:root:Trial completed: MAE=7255.9034971548035
INFO:root:Trial completed: MAE=6232.872586430039
INFO:root:Trial completed: MAE=7639.050539565518
INFO:root:Trial completed: MAE=5744.810384769555
INFO:root:Trial completed: MAE=8749.326552413439
INFO:root:Trial completed: MAE=6474.410012483757
INFO:root:Trial completed: MAE=5320.662297679305
INFO:root:Trial completed: MAE=5269.991722961467


No best pipeline found. Check the trial setup and execution.


## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [77]:
import optuna
import pickle
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

In [68]:
!pip install optuna-integration[xgboost]

  Using cached optuna_integration-4.0.0-py3-none-any.whl.metadata (11 kB)
Using cached optuna_integration-4.0.0-py3-none-any.whl (96 kB)


In [80]:
def objective(trial):
    # Hyperparameters
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42  
    }

    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    regressor = XGBRegressor(**params)

    pipeline = Pipeline([
        ('encoder', encoder),
        ('regressor', regressor)
    ])

    
    pipeline.fit(
        X_train, y_train,
        regressor__eval_set=[(X_valid, y_valid)],
        regressor__eval_metric='mae',
        regressor__early_stopping_rounds=20,
        regressor__callbacks=[XGBoostPruningCallback(trial, "validation_0-mae")]
    )

    preds = pipeline.predict(X_valid)
    mae = mean_absolute_error(y_valid, preds)
    return mae

sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100, timeout=300)

# Output results
if study.best_trial:
    best_pipeline = study.user_attrs.get("best_pipeline", None)
    if best_pipeline:
        with open('best_model.pkl', 'wb') as f:
            pickle.dump(best_pipeline, f)
    print(f"Best MAE: {study.best_value}")
    print("Best Hyperparameters:", study.best_trial.params)
else:
    print("No best pipeline found. Check the trial setup and execution.")


[W 2024-10-25 23:59:01,837] Trial 0 failed with parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'min_child_weight': 3, 'reg_alpha': 0.15601864044243652, 'reg_lambda': 0.15599452033620265} because of the following error: TypeError("XGBModel.fit() got an unexpected keyword argument 'eval_metric'").
Traceback (most recent call last):
  File "c:\Users\siver\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\siver\AppData\Local\Temp\ipykernel_1056\1996139088.py", line 22, in objective
    pipeline.fit(
  File "c:\Users\siver\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\siver\anaconda3\Lib\site-packages\sklearn\pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"]

TypeError: XGBModel.fit() got an unexpected keyword argument 'eval_metric'

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [82]:
pip install plotly



SyntaxError: invalid syntax (1588724634.py, line 1)

In [83]:
import optuna.visualization as ov


ov.plot_optimization_history(study).show()

ov.plot_parallel_coordinate(study).show()

ov.plot_param_importances(study).show()


[W 2024-10-26 00:03:02,154] There are no complete trials.


[W 2024-10-26 00:03:02,547] Your study does not have any completed trials.


[W 2024-10-26 00:03:02,553] Study instance does not contain completed trials.


## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:


with open('best_model.pkl', 'rb') as f:
    best_model = pickle.load(f)


predictions = best_model.predict(X_test)
mae_test = mean_absolute_error(y_test, predictions)
print("MAE en el conjunto de test:", mae_test)


Modelo	MAE en Validación
Baseline XGBoost	---
XGBoost + Const	    ---
XGBoost + Optuna	---
XGBoost + Prunning	---

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>